In [1]:
import re
import pandas as pd
import requests
import pdfplumber

In [62]:
def extrair(x):
  
    with pdfplumber.open(x) as y:
      PDF = y.pages[0]
      PDF.chars[0]
      pdf = PDF.extract_text().split("\n")

    tira_rep = 0
    for x in range(0, len(pdf)):
        #Nome do contribuinte
        if pdf[x] == 'Contribuinte D.A.M.':
            nome = pdf[x+1].upper()
            nome = re.sub('[^A-Z]', ' ', nome).strip(" ")
            
        #CPF/CNPJ  //  Inscrição/CMC  //  Tributos  //  Referência  //  Vencimento
        if pdf[x][:8] == 'CPF/CNPJ':
            if tira_rep == 0:
                tira_rep = 1
                dados = pdf[x+1].split(" ")
                cpf_cnpj = dados[0]
                inscricao_CMC = dados[1]
                tributos = dados[2] + " " + dados[3]
                referencia = dados[4]
                vencimento = dados[5]
                
        #Logradouro  //  Número  //  CEP
        if pdf[x][:10] == 'Logradouro':
            dados = pdf[x].split(" ")
            logradouro = pdf[x][12:]
            logradouro = re.sub('[0-9]', ' ', logradouro).strip(" ")[:-4]

            numero = pdf[x][12:]
            numero = re.sub('[^0-9]', ' ', numero).strip(" ").split(" ")[0]
            
            cep = pdf[x][12:]
            cep = re.sub('[^0-9]', ' ', cep).strip(" ").split(" ")[-1]
        
        #Bairro  //  Complemento
        if pdf[x][:6] == 'Bairro':
          bc = pdf[x][8:].upper().split("COMPLEMENTO:")

          bairro = bc[0]
          complemento = bc[1]
        
        #Área terreno  //  Área Total Construída  //  Área Construída Unidade
        if pdf[x][:13] == 'Área Terreno:':

          ara_terreno = pdf[x][14:].split(" ")[0]

          area_total_construida = pdf[x][14:].split("Área Total Construída: ")[1].split(" ")[0]

          ara_construida_unidade = pdf[x][14:].split(" ")[-1]
         
        #Valor Venal Terreno  //  Valor Venal Construção
        if pdf[x][:20] == 'Valor Venal Terreno:':

          valor_venal_terreno = pdf[x][21:].split(" ")[0]

          valor_venal_construcao = pdf[x].split(" ")[-1]
        
        #Valor Venal Imóvel  //  Base de Cálculo  //  Alíquota
        if pdf[x][:19] == 'Valor Venal Imóvel:':
          valor_venal_imovel = pdf[x][20:].split(" ")[0]

          base_de_calculo = pdf[x][14:].split("Base de Cálculo: ")[1].split(" ")[0]

          aliquota = pdf[x][14:].split(" ")[-1]
         
        #Total a pagar
        if pdf[x][0:len('Total a Pagar: ')] == 'Total a Pagar: ':
            total_a_pagar = pdf[x].split(" ")[-1]
            
    #codigo de barras     
    codBarras = pdf[-1]
    
    dados_para_df = {"Nome do contribuinte"   : [nome],
                     "CPF/CNPJ"               : [cpf_cnpj],
                     "Inscrição/CMC"          : [inscricao_CMC],
                     "Tributos"               : tributos,
                     "Referência"             : [referencia],
                     "Vencimento"             : [vencimento],
                     "Logradouro"             : [logradouro],
                     "Número"                 : [numero],
                     "CEP"                    : [cep],
                     "Bairro"                 : [bairro],
                     "Complemento"            : [complemento],
                     "Área terreno"           : [ara_terreno],
                     "Área Total Construída " : [area_total_construida],
                     "Área Construída Unidade": [ara_construida_unidade],
                     "Valor Venal Terreno"    : [valor_venal_terreno],
                     "Valor Venal Construção" : [valor_venal_construcao],
                     "Valor Venal Imóvel"     : [valor_venal_imovel],
                     "Base de Cálculo"        : [base_de_calculo],
                     "Alíquota"               : [aliquota],
                     "Total a pagar"          : [total_a_pagar],
                     "Códigos de barras"      : [codBarras]}
    
    df = pd.DataFrame(dados_para_df, index = ["DADOS"]).T
    
    return df

In [64]:
extrair("arquivo/Rua Ângelo Martins, 445 1-10.pdf")

,DADOS
Nome do contribuinte,MANOEL CORREIA TELLES
CPF/CNPJ,002.643.184-04
Inscrição/CMC,000000000120433
Tributos,I.P.T.U. 2021
Referência,1/10
Vencimento,31/03/2021
Logradouro,RUA - ÂNGELO MARTINS
Número,00445
CEP,57030592
Bairro,PONTA DA TERRA
